Here we will deal with the coding we have using spotify's api and returning a song, and make small functions out of it, so we can call them in the product prototype, which is the last thing on this notebook

In [1]:
import spotipy # install if needed
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
#Initialize SpotiPy with user credentias
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(
    client_id="6582a8f807004727b3bc8a7c281ae02c",
    client_secret="c38fc3efa7f74def856cb96d000fd6ea"))

In [3]:
#create a list of playlists url's:

list_url = ["https://open.spotify.com/playlist/13qezNPeoOZbnpgYdvfXzw", "https://open.spotify.com/playlist/7vse1mpYM4ZZMAnFjtDmxn", "https://open.spotify.com/playlist/5BrB90Tlv372JYksHKEUUC", "https://open.spotify.com/playlist/4gGVp5hZBGMT5u8Umwy71i", "https://open.spotify.com/playlist/69IzgRxJudWrroRkFz6rqD", "https://open.spotify.com/playlist/7Bz2d7LtQAA4sdqjK54ihN", "https://open.spotify.com/playlist/3jN4ozUr5QnxLJAuWSOz8L", "https://open.spotify.com/playlist/7nb8QoKz6X7j43a8JLDWWa", "https://open.spotify.com/playlist/0Elk90H3u3Pii6HmU7uwDQ", "https://open.spotify.com/playlist/5jL36juHp2hI5p4w7Otkp8", "https://open.spotify.com/playlist/1FNxkddOTy0dEQjzO605Mu", "https://open.spotify.com/playlist/71AoD5gr7igctQEioLcrgs", "https://open.spotify.com/playlist/28PCyERc69PVABnVlhzCzK", "https://open.spotify.com/playlist/1AHXVPpZj0ttq2kEHfRrBz", "https://open.spotify.com/playlist/7GQpGQ7mo1qNgjRYzTIROw", "https://open.spotify.com/playlist/1n6ApCcBNm1LlObavhViZV", "https://open.spotify.com/playlist/52p0TeTGoMt2bdYaCgDgGi", "https://open.spotify.com/playlist/1vs8WUfA0QQO4V0rtVpFKX"]


In [4]:
#getting content of the playlists with "next"
playlists = []
for i in list_url:
    track = sp.user_playlist_tracks("spotify", i)
    playlists.append(track)
    while track['next']:
        track = sp.next(track)
        playlists.append(track) 
print(len(playlists))


37


In [5]:
songs = []
for i in range(len(playlists)):
    for song in playlists[i]["items"]:
        songs.append(song["track"]["name"])
print(len(songs))
    

2977


In [6]:
len(set(songs))

2619

In [7]:
def get_uri():
    uris = []
    for i in range(len(playlists)):
        for uri in playlists[i]["items"]:
            uris.append(uri["track"]["uri"])
    return uris

In [8]:
uris = get_uri()
len(uris)

2977

In [9]:
def get_artist():
    artists = []
    for i in range(len(playlists)):
        for artist in playlists[i]["items"]:
            artists.append(artist["track"]["artists"][0]["name"])
    return artists    

In [10]:
artists = get_artist()
len(artists)

2977

In [11]:
def get_audio_feat():
    audio_feat = []
    for i in uris:
        audio_feat.append(sp.audio_features(i))
    return audio_feat    

In [12]:
audio_feat = get_audio_feat()
len(audio_feat)

2977

In [ ]:
# now i have: a list with song names, and a list with their audio features. Lets put them in a df

In [13]:
def df_songs_features():
    danceability = []
    for song in audio_feat:
        for i in song:
            danceability.append(i["danceability"])
    loudness = []
    for song in audio_feat:
        for i in song:
            loudness.append(i["loudness"])
    speechiness = []
    for song in audio_feat:
        for i in song:
            speechiness.append(i["speechiness"])
    acousticness = []
    for song in audio_feat:
        for i in song:
            acousticness.append(i["acousticness"])
    energy = []
    for song in audio_feat:
        for i in song:
            energy.append(i["energy"])
    instrumentalness = []
    for song in audio_feat:
        for i in song:
            instrumentalness.append(i["instrumentalness"])
    tempo = []
    for song in audio_feat:
        for i in song:
            tempo.append(i["tempo"])
            
    import pandas as pd

    songs_features_dict = {
        "name_of_song" : songs,
        "artist": artists,
        "danceability": danceability,
        "energy": energy,
        "loudness": loudness,
        "speechiness": speechiness,
        "acousticness": acousticness,
        "instrumentalness": instrumentalness,
        "tempo": tempo,
        "uri": uris
    }

    songs_features = pd.DataFrame(songs_features_dict)
    return songs_features    

In [14]:
songs_features = df_songs_features()

In [15]:
songs_features.head()

,name_of_song,artist,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,uri
0,Bring Me To Life,Evanescence,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,spotify:track:0COqiPhxzoWICwFCS4eZcp
1,Like The Way I Do,Melissa Etheridge,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,spotify:track:5cj6toDqMzkbpgkGoRav0S
2,The Chain - 2004 Remaster,Fleetwood Mac,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,spotify:track:5e9TFTbltYBg2xThimr0rU
3,Stars,Grace Potter & The Nocturnals,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,spotify:track:6s7sq2Quh4TxHRfdk6eFxk
4,White Rabbit,Jefferson Airplane,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,spotify:track:4vpeKl0vMGdAXpZiQB2Dtd


Now, we will work on clustering the songs

In [16]:
from sklearn.preprocessing import StandardScaler

In [17]:
list_audio_feat = ["danceability", "energy", "loudness", "speechiness", "acousticness", "instrumentalness", "tempo"]
X = songs_features[list_audio_feat]

In [18]:
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo
0,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612
1,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426
2,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553
3,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262
4,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798


In [ ]:
X.describe()

In [19]:
scaler = StandardScaler()

In [20]:
X_scaled = scaler.fit_transform(X)

In [21]:
# now I have an array of the features of each song, fitted and transformed, meaning they had been changed according to mean and std deviation
X_scaled

array([[-1.69993753,  1.71514086,  1.37586362, ..., -0.98105468,
        -0.21371423, -0.88248963],
       [ 0.05771423,  1.63385885,  0.01121345, ..., -0.91771593,
        -0.21371696, -0.64786768],
       [-0.33216489,  0.40984739, -0.43358526, ..., -0.97514658,
        -0.2067262 ,  1.07812213],
       ...,
       [ 0.09606299,  1.80120417,  1.41126728, ..., -0.6516866 ,
        -0.2133764 ,  0.33162904],
       [ 0.07049715,  0.84016392,  0.51877319, ..., -0.9622742 ,
        -0.21373179,  1.65169602],
       [ 0.99086752, -0.67072524, -0.11784899, ...,  1.60923264,
        -0.21373179, -0.42095857]])

In [22]:
#we use Kmeans to cluster
from sklearn.cluster import KMeans

In [23]:
#initializing
my_kmeans = KMeans(random_state=1234)

In [24]:
#store information from my model inside this kmeans
my_kmeans.fit(X_scaled)
#these are the default parameters used:

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=8, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=1234, tol=0.0001, verbose=0)

In [25]:
# Predicting the clusters. It will gives us an array of clusters:
clusters = my_kmeans.predict(X_scaled)
clusters

array([0, 4, 1, ..., 7, 7, 3])

In [26]:
import pandas as pd

In [27]:
#I want to analyze and explore them in a DF because it is easier
clusters = pd.Series(clusters)
clusters

0       0
1       4
2       1
3       1
4       2
       ..
2972    0
2973    7
2974    7
2975    7
2976    3
Length: 2977, dtype: int32

In [28]:
clusters.value_counts().sort_values()

5     76
7    274
2    308
6    360
1    404
0    429
3    443
4    683
dtype: int64

In [30]:
# I will add a column in my audio features (X) dataframe telling me in which cluster each song belongs
X["cluster"] = clusters

C:\Users\Mariana\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [31]:
X.head()

,danceability,energy,loudness,speechiness,acousticness,instrumentalness,tempo,cluster
0,0.331,0.943,-3.188,0.0698,0.00721,0.000002,94.612,0
1,0.606,0.926,-7.428,0.0437,0.02640,0.000002,101.426,4
2,0.545,0.670,-8.810,0.0496,0.00900,0.000822,151.553,1
3,0.432,0.496,-6.918,0.0274,0.18100,0.000783,140.262,1
4,0.551,0.293,-12.503,0.0272,0.48600,0.000273,106.798,2


In [ ]:
## ideally we explore clusters, but let's move on for now.
## let's work with getting a song from user, requestiong its audio features to spotify, and assigning it to a cluster
# def song_to_lookfor():
    #user_input = input("Tell me the name of a song you like, I'll give you a suggestion you might like: ")
    #input_song = user_input
    #result = sp.search(input_song)
    #song_uri = result["tracks"]["items"][0]["uri"]
    #return song_uri
    

In [ ]:
## I extract the URI, because that is what I'll use to get the audio features
#def extract_uri():
    #song_uri = result["tracks"]["items"][0]["uri"]
    #return song_uri

song_uri = extract_uri()

In [32]:
## let's work with getting a song from user, requestiong its audio features to spotify, and assigning it to a cluster

def get_uris(song_name):
    result = sp.search(song_name)
    song_uri = result["tracks"]["items"][0]["uri"]
    return song_uri

In [45]:
## Now I get the audio features from that song and make it a df
def extract_af(song_uri):
    song_af = sp.audio_features(song_uri)
    song_af_dict = song_af[0]
    song_df = pd.DataFrame(song_af_dict,index=[0])
    song_df.drop(columns=["key", "mode", "liveness", "valence", "type", "id", "uri", "track_href", "analysis_url", "duration_ms", "time_signature"], inplace=True)
    return song_df

extract_af()

In [56]:
# Scale (fit and transform), and predict!
def assign_cluster(song_features):
    song_df_scaled = scaler.transform(song_features)
    song_features = pd.DataFrame(song_df_scaled)
    song_features["cluster"] = my_kmeans.predict(song_df_scaled)
    return song_features

In [35]:
len(X["tempo"])

2977

In [36]:
len(songs)

2977

In [52]:
X["name_of_song"] = songs
X["artist"] = artists

In [53]:
def recommendation(song_features_and_cluster):
    cluster_to_lookfor = song_features_and_cluster["cluster"][0]
    same_cluster_df = X.loc[X['cluster'] == cluster_to_lookfor]
    choice = same_cluster_df.sample()
    return "Try listening to " + choice.iloc[0,8] + ", by " + choice.iloc[0,9]

## Joining everything

In [40]:
import requests
from bs4 import BeautifulSoup

In [41]:
url = "https://www.billboard.com/charts/hot-100"
billboard = requests.get(url)
soup = BeautifulSoup(billboard.content, "html.parser")

In [42]:
songs_billboard = soup.select("span.chart-element__information__song.text--truncate.color--primary")
songs_top100 = []

for s in songs_billboard:
    songs_top100.append(s.get_text())

In [57]:
import random

def song_recommender():
    user_input = input("Tell me the name of a song you like, and I'll give you a suggestion you might enjoy: ")
    if user_input in songs_top100:
        return "You like trendy songs! I'll give you another one: " + random.choice(songs_top100)
    else:
        uri = get_uris(user_input)
        features = extract_af(song_uri=uri)
        features_and_cluster = assign_cluster(song_features=features)
        return recommendation(song_features_and_cluster=features_and_cluster)

        

**Let's have a preview of how it works!**

In [58]:
song_recommender()

Tell me the name of a song you like, and I'll give you a suggestion you might enjoy: breakaway


'Try listening to Weak, by SWV'